In [6]:
# ─── Cell 1: Add scripts folder to path & import modules ────────────────────────
import sys
from pathlib import Path

# 1) Locate project root (one level up from this notebook’s folder)
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    # In a notebook __file__ won’t exist, so fall back
    project_root = Path.cwd().parent

# 2) Point at your scripts folder
scripts_dir = project_root / "scripts_optimisation"
assert scripts_dir.exists(), f"Can't find scripts_optimisation at {scripts_dir}"
sys.path.insert(0, str(scripts_dir))

# 3) Import your PV / battery / financial / objective modules
from pv_simulate import simulate_multi_year_pv
from battery     import simulate_battery_dispatch
from financial   import compute_financials
from objective   import evaluate_solution

# 4) Import optimisation & data libs
import numpy as np
import pandas as pd
from pymoo.core.problem         import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination          import get_termination
from pymoo.optimize             import minimize

print("✅ All modules imported successfully!")


✅ All modules imported successfully!


In [8]:
# ─── Cell 2: Configuration & NSGA-II Run ────────────────────────────────────────

import numpy as np
import pandas as pd
from pymoo.core.problem         import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination          import get_termination
from pymoo.optimize             import minimize

# 1) Paths & Parameters ---------------------------------------------------------
data_dir        = project_root / "data"
outputs_dir     = project_root / "outputs_optimisation"
outputs_dir.mkdir(exist_ok=True)

ROOF_PARAMS     = [{
    'name':               'roof1',
    'system_capacity_kw': 10.0,
    'tilt':               10.0,
    'azimuth':            18.0,
    'shading':            43.5
}]

WEATHER_FILES   = [
    data_dir / "Bonfire_2025.epw",
    data_dir / "Bonfire_2040.epw",
    data_dir / "Bonfire_2050.epw"
]
REPEATS_PER_FILE = 10
DEMAND_CSV       = data_dir / "PV_Generation_excel.csv"

def load_demand_profile(path: str, repeats: int) -> pd.Series:
    """
    Load one‐year demand and tile it for the full horizon.
    Expects a CSV with a datetime column 'Date and Time' and
    a numeric 'Consumed Power' column (kWh).
    """
    df = pd.read_csv(path, parse_dates=['Date and Time'], index_col='Date and Time')
    base = df['Consumed Power']
    tiled = pd.concat([base.copy() for _ in range(repeats)], ignore_index=True)
    idx   = pd.date_range(base.index[0], periods=len(tiled), freq='30min')
    tiled.index = idx
    return tiled

# 2) Precompute PV & demand for 30 years ----------------------------------------
print("Building 30-year PV profile…")
pv_profile = simulate_multi_year_pv(
    WEATHER_FILES,
    ROOF_PARAMS,
    repeats_per_file=REPEATS_PER_FILE
)

print("Building 30-year demand profile…")
demand_profile = load_demand_profile(
    str(DEMAND_CSV),
    repeats=REPEATS_PER_FILE * len(WEATHER_FILES)
)

# 3) Define the multi-objective problem ----------------------------------------
class SolarBatteryProblem(Problem):
    def __init__(self):
        super().__init__(
            n_var=1,        # battery_kwh
            n_obj=2,        # –IRR, NPC
            n_constr=0,
            xl=np.array([0.0]),
            xu=np.array([150.0])
        )

    def _evaluate(self, X, out, *args, **kwargs):
        F = []
        for row in X:
            batt_kwh = float(row[0])
            params = {
                'pv_kw':       ROOF_PARAMS[0]['system_capacity_kw'],
                'battery_kwh': batt_kwh
            }
            # evaluate_solution returns [-IRR, NPC]
            F.append(evaluate_solution(params, pv_profile, demand_profile))
        out["F"] = np.array(F)

# 4) Run NSGA-II ---------------------------------------------------------------
problem     = SolarBatteryProblem()
algorithm   = NSGA2(pop_size=40)
termination = get_termination("n_gen", 50)

print("Running NSGA-II… this may take a few minutes")
res = minimize(
    problem,
    algorithm,
    termination,
    seed=42,
    verbose=True
)

# 5) Extract & save the Pareto front -----------------------------------------
battery_sizes = res.X.flatten()
pareto_F      = res.F
irr_vals      = -pareto_F[:, 0]   # undo the negation
npc_vals      = pareto_F[:, 1]

df = pd.DataFrame({
    'battery_kwh': battery_sizes,
    'IRR':          irr_vals,
    'NPC':          npc_vals
})

out_csv = outputs_dir / "pareto_solutions.csv"
df.to_csv(out_csv, index=False)
print(f"✅ Optimisation complete. Results saved to {out_csv}")


Building 30-year PV profile…


Exception: pvwattsv8 execution error.
	exec fail(pvwattsv8): could not open file for reading: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2040.epw

